### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy import plotting
import seaborn as sns
import pandas as pd
from scipy import stats
from neuropy.analyses.oscillations import detect_ripple_epochs
from tqdm.notebook import tqdm
import subjects

### Detect ripples for multiple sessions together

In [ ]:
sessions = subjects.sdrol.ratN_2019_10_19

for sub, sess in enumerate(tqdm(sessions)):

    try:
        artifact = sess.artifact
    except:
        artifact = None

    # channels = sess.ripple.metadata['channels']
    signal = sess.eegfile.get_signal()
    ripples = detect_ripple_epochs(
        signal,
        probegroup=sess.probegroup,
        freq_band=(125, 250),
        thresh=(2.5, 30),
        edge_cutoff=0.5,
        sigma=0.0125,
        ignore_epochs=artifact,
    )

    ripples.save(sess.filePrefix.with_suffix('.ripple.npy'))
    # ripples.save(sess.filePrefix.with_suffix(".ripple.no_merge.npy"))


### Ripple (125-250) band comparison with EMG band (>300 Hz)
- This is to test a potential chewing artifact rejection algorithm.
- Basically based on the 1/f law the power in first harmonic (250-500) of ripple band should be smaller than at fundamental band. If the power is higher in >300 Hz then it is potentially contaminated by chewing/muscle artifact.

In [ ]:
from neuropy.utils.signal_process import filter_sig, hilbertfast
from scipy.ndimage import gaussian_filter1d

dt = 1/1250
smooth = lambda x: gaussian_filter1d(x, sigma=0.0125 / dt, axis=-1)

sess = subjects.sdrol.ratU_2021_08_09[0]
ripple_channels = sess.ripple.metadata["channels"]

t1 = 226 * 60 + 20
lfp = sess.eegfile.get_signal(58, t_start=t1, t_stop=t1 + 3)

f1,f2=125,250

rpl_lfp = filter_sig.bandpass(lfp, lf=f1, hf=f2)
fharm_lfp = filter_sig.bandpass(lfp, lf=2*f1, hf=2*f2)
emg_lfp = filter_sig.bandpass(lfp, lf=300, hf=600)

corr_traces = sess.eegfile.get_signal(t_start=t1,t_stop=t1+5).traces

ltriang = np.tril_indices(corr_traces.shape[0],k=-1)

mov_corr=[]
for i in range(0,corr_traces.shape[1],50): 
    mov_corr.append(np.corrcoef(corr_traces[:,i:i+50])[ltriang].mean())



In [ ]:
_,ax = plt.subplots()

sigs = [rpl_lfp,fharm_lfp,emg_lfp]
colors = ['r','g','k']

for i,sig in enumerate(sigs):
    trace = smooth(np.abs(hilbertfast(sig.traces[0])))
    ax.plot(trace,color=colors[i])

ax.plot(mov_corr)

In [ ]:
from numpy.lib.stride_tricks import sliding_window_view

a = sliding_window_view(corr_traces,50,axis=-1)